In [1]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from more.datasets import ModulationDataSets
from more.network import MoreNet,init_weights
from more.train import train_center
from utils.centerloss import CenterLoss
import datetime
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
train_path = r'E:\train.h5'
test_path = r'E:\test.h5'
train_dataset = ModulationDataSets(train_path)
test_dataset = ModulationDataSets(test_path)
label_name = train_dataset.get_labels_name();

In [3]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)

In [4]:
net = MoreNet()
X = torch.rand(size=(1, 1, 196, 64), dtype=torch.float32)
net(X,True)

E:\workspace\DeepMore\more\network.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return x.detach(), F.softmax(y)


(tensor([[0.0000, 0.0067]]),
 tensor([[0.1996, 0.0826, 0.1262, 0.1612, 0.0693, 0.1069, 0.1507, 0.1036]],
        grad_fn=<SoftmaxBackward0>))

In [5]:
#net.load_state_dict(torch.load('basic-8.parm'))

In [6]:
#torch.save(net.state_dict(),"basic-8.parm")

In [7]:
init_weights(net)

In [8]:
loss= nn.CrossEntropyLoss()
center_loss=CenterLoss(num_classes=train_dataset.get_numclasses(),feat_dim=2,use_gpu=True)
lr=0.001
lr_center=0.5
opt_model=torch.optim.SGD(net.parameters(),lr,weight_decay=5e-04, momentum=0.9)
opt_center=torch.optim.SGD(center_loss.parameters(),lr_center)

scheduler = torch.optim.lr_scheduler.StepLR(opt_model,step_size=20,gamma=0.5)

net=net.to("cuda:0")

In [ ]:
start_time = time.time()

for epoch in range(150):
    print("==> Epoch {}/{}".format(epoch+1,50))

    train_center(net, loss, center_loss, opt_model, opt_center, train_loader, True, train_dataset.get_numclasses(), epoch)
    scheduler.step()
    # if  (epoch+1) % 10 == 0 or (epoch+1) == 50:
    #     print("==> Test")
    #     acc, err = test(model, testloader, use_gpu, dataset.num_classes, epoch)
    #     print("Accuracy (%): {}\t Error rate (%): {}".format(acc, err))

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

==> Epoch 1/50


E:\workspace\DeepMore\utils\centerloss.py:35: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1055.)
  distmat.addmm_(1, -2, x, self.centers.t())


Batch 50/625	 Loss 2.082170 (2.128346) XentLoss 2.082142 (2.080015) CenterLoss 0.000028 (0.048331)
Batch 100/625	 Loss 2.078378 (2.104566) XentLoss 2.078374 (2.080387) CenterLoss 0.000004 (0.024179)
Batch 150/625	 Loss 2.075376 (2.096395) XentLoss 2.075373 (2.080275) CenterLoss 0.000003 (0.016120)
Batch 200/625	 Loss 2.073854 (2.092175) XentLoss 2.073848 (2.080084) CenterLoss 0.000006 (0.012091)
Batch 250/625	 Loss 2.076496 (2.089738) XentLoss 2.076494 (2.080064) CenterLoss 0.000003 (0.009674)
Batch 300/625	 Loss 2.083633 (2.088121) XentLoss 2.083632 (2.080059) CenterLoss 0.000000 (0.008062)
Batch 350/625	 Loss 2.073328 (2.087058) XentLoss 2.073326 (2.080147) CenterLoss 0.000002 (0.006911)
Batch 400/625	 Loss 2.077356 (2.086287) XentLoss 2.077348 (2.080240) CenterLoss 0.000008 (0.006047)
Batch 450/625	 Loss 2.079896 (2.085544) XentLoss 2.079881 (2.080167) CenterLoss 0.000015 (0.005376)
Batch 500/625	 Loss 2.087861 (2.085037) XentLoss 2.087856 (2.080198) CenterLoss 0.000004 (0.004840)
B

In [ ]:
torch.save(net.state_dict(),"basic-basic.parm")